In [44]:
import pickle

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

In [45]:
import warnings
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", RuntimeWarning)

In [46]:
ptv_result = [0] * 9

In [ ]:
with open(f'ptv_cbi_q_10.pickle', mode='rb') as fp:
    ptv_result[0] = pickle.load(fp)
with open(f'ptv_cbi_q_20.pickle', mode='rb') as fp:
    ptv_result[1] = pickle.load(fp)
# with open(f'ptv_cbi_q_30.pickle', mode='rb') as fp:
#     ptv_result[2] = pickle.load(fp)
with open(f'ptv_cbi_q_30_high.pickle', mode='rb') as fp:
    ptv_result[2] = pickle.load(fp)
with open(f'ptv_rbi_q_10_1.pickle', mode='rb') as fp:
    ptv_result[3] = pickle.load(fp)
with open(f'ptv_rbi_q_10_2.pickle', mode='rb') as fp:
    ptv_result[4] = pickle.load(fp)
with open(f'ptv_rbi_q_20_1.pickle', mode='rb') as fp:
    ptv_result[5] = pickle.load(fp)
with open(f'ptv_rbi_q_20_2.pickle', mode='rb') as fp:
    ptv_result[6] = pickle.load(fp)
with open(f'ptv_rbi_q_30_1.pickle', mode='rb') as fp:
    ptv_result[7] = pickle.load(fp)
with open(f'ptv_rbi_q_30_2.pickle', mode='rb') as fp:
    ptv_result[8] = pickle.load(fp)

In [ ]:
for i in range(9):
    ptv_result[i]['r'] = np.sqrt(ptv_result[i]['x']**2 + ptv_result[i]['y']**2)

In [ ]:
px2mm = 1000 / np.sqrt((498 - 488)**2 + (391 - 646)**2)
fps = 25

In [ ]:
res_sort = [0] * 9

for i in range(9):
    res_sort[i] = ptv_result[i].sort_values(by='r', ascending=True).values

In [ ]:
_iter = 50
res = [0]*9
res2 = [0]*9

In [ ]:
for c in range(9):
    r_dr_err = np.zeros((5001, 3))
    r_dr_in = np.zeros((1, 2))
    r_dr_err[:, :] = np.nan
    r_dr_in[:, :] = np.nan

    for i in tqdm(range(0, 5001, _iter)):
        res_sort_l = res_sort[c][np.where(res_sort[c][:, 4] >= i/10)]
        res_sort_lr = res_sort_l[np.where(res_sort_l[:, 4] < (i+_iter)/10)]
        columns = ['x', 'y', 'dx', 'dy', 'r', 'theta', 'dr']
        df = pd.DataFrame(data=res_sort_lr, columns=columns, dtype='float')
        q1 = df['dr'].quantile(0.25)
        q2 = df['dr'].quantile(0.75)
        iqr = q2 - q1

        upper = q2 + 1.5 * iqr
        bottom = q1 - 1.5 * iqr
        _in = df[(df['dr'] >= bottom) & (df['dr'] <= upper)].values
        tmp = np.array([_in[:, 4], _in[:, 6]]).T
        r_dr_in = np.vstack([r_dr_in, tmp])

        r_dr_err[i, 0] = i
        r_dr_err[i, 1] = np.mean(tmp, axis=0)[1]
        r_dr_err[i, 2] = np.std(tmp, axis=0)[1]

    r_dr_err = r_dr_err[::_iter]
    res[c] = r_dr_err
    res2[c] = r_dr_in

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
ax.scatter(res[0][:-25, 0]*px2mm/10, res[0][:-25, 1]*px2mm*fps)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_yscale("log")
plt.xlabel("r [m]")
plt.ylabel("u_r [m/s]")
ax.scatter(res[0][:-25, 0]*px2mm/10/1e3, res[0][:-25, 1]*px2mm*fps/1e3)
ax.scatter(res[1][:-25, 0]*px2mm/10/1e3, res[1][:-25, 1]*px2mm*fps/1e3)
ax.scatter(res[2][:-25, 0]*px2mm/10/1e3, res[2][:-25, 1]*px2mm*fps/1e3)
plt.show()
fig.savefig("semi-log.png")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_xscale("log")
plt.xlabel("r [m]")
plt.ylabel("u_r [m/s]")
ax.scatter(res[0][:-25, 0]*px2mm/10/1e3, res[0][:-25, 1]*px2mm*fps/1e3, c='b')
ax.scatter(res[1][:-25, 0]*px2mm/10/1e3, res[1][:-25, 1]*px2mm*fps/1e3, c='g')
ax.scatter(res[2][:-25, 0]*px2mm/10/1e3, res[2][:-25, 1]*px2mm*fps/1e3, c='r')
ax.vlines(0.4, -0.15, 0.45, colors='k')
ax.vlines(1.5, -0.15, 0.45, colors='k')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_xscale("log")
ax.set_yscale("log")
plt.xlabel("r [m]")
plt.ylabel("u_r [m/s]")
ax.scatter(res[0][:-25, 0]*px2mm/10/1e3, res[0][:-25, 1]*px2mm*fps/1e3, c='b')
ax.scatter(res[1][:-25, 0]*px2mm/10/1e3, res[1][:-25, 1]*px2mm*fps/1e3, c='g')
ax.scatter(res[2][:-25, 0]*px2mm/10/1e3, res[2][:-25, 1]*px2mm*fps/1e3, c='r')
ax.vlines(0.4, -0.15, 0.45, colors='k')
ax.vlines(1.5, -0.15, 0.45, colors='k')
plt.show()
fig.savefig("loglog2.png")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_xscale("log")
ax.set_yscale("log")
plt.xlabel("r [m]")
plt.ylabel("u_r [m/s]")
# plt.xlim(0.1, 10)
ax.scatter(res[0][20:-25, 0]*px2mm/10/1e3, res[0][20:-25, 1]*px2mm*fps/1e3, c='b')
ax.scatter(res[1][20:-25, 0]*px2mm/10/1e3, res[1][20:-25, 1]*px2mm*fps/1e3, c='g')
ax.scatter(res[2][20:-25, 0]*px2mm/10/1e3, res[2][20:-25, 1]*px2mm*fps/1e3, c='r')
ax.plot(np.linspace(0.4, 1.4, 100), 1 / np.linspace(0.4, 1.4, 100), c='k', linestyle='-')
ax.plot(np.linspace(0.4, 1.4, 100), 0.4 / np.linspace(0.4, 1.4, 100)**2, c='k', linestyle='--')
plt.show()
fig.savefig("loglog3.png")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_xscale("log")
ax.set_yscale("log")
plt.xlabel("r [m]")
plt.ylabel("u_r [m/s]")
ax.scatter(res[0][20:-25, 0]*px2mm/10/1e3, res[0][20:-25, 1]*px2mm*fps/1e3, c='b', zorder=10)
ax.plot(np.linspace(0.4, 2, 100), 1 / np.linspace(0.4, 2, 100) * 0.11, c='k', linestyle='-')
ax.plot(np.linspace(0.4, 2, 100), 1 / np.linspace(0.4, 2, 100)**2 * 0.11, c='k', linestyle='--')
plt.show()
fig.savefig("loglog4.png")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_xscale("log")
ax.set_yscale("log")
plt.xlabel("r [m]")
plt.ylabel("u_r [m/s]")
ax.scatter(res[1][20:-25, 0]*px2mm/10/1e3, res[1][20:-25, 1]*px2mm*fps/1e3, c='g', zorder=10)
ax.plot(np.linspace(0.4, 2, 100), 1 / np.linspace(0.4, 2, 100) * 0.14, c='k', linestyle='-')
ax.plot(np.linspace(0.4, 2, 100), 1 / np.linspace(0.4, 2, 100)**2 * 0.13, c='k', linestyle='--')
plt.show()
fig.savefig("loglog5.png")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_xscale("log")
ax.set_yscale("log")
plt.xlabel("r [m]")
plt.ylabel("u_r [m/s]")
ax.scatter(res[2][20:-25, 0]*px2mm/10/1e3, res[2][20:-25, 1]*px2mm*fps/1e3, c='r', zorder=10)
ax.plot(np.linspace(0.4, 2, 100), 1 / np.linspace(0.4, 2, 100) * 0.15, c='k', linestyle='-')
ax.plot(np.linspace(0.4, 2, 100), 1 / np.linspace(0.4, 2, 100)**2 * 0.15, c='k', linestyle='--')
plt.show()
fig.savefig("loglog6.png")